In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)
import timeit

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

from vivarium_research_prl.utils import sizemb
from vivarium_research_prl import datatypes
# from vivarium_research_prl.noise import corruption, fake_names, noisify
# from vivarium_research_prl.find_kids import datasets, noisify_data

!date
!whoami
!uname -a
!pwd

Tue 21 Mar 2023 01:35:03 PM PDT
ndbs
Linux int-slurm-sarchive-p0011 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [11]:
import os

# Find data

Run of 18 seeds, reflecting a population of 18 million, from Rajan on 3/17/2023:
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19
```

In [6]:
project_output_dir = \
    '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = (
    'results/v1.2_parallelize_business_names'
    '/united_states_of_america/2023_03_16_17_21_09'
    '/final_results/2023_03_17_15_28_19'
)
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -halt $output_dir

total 44K
drwxrwsr-x  5 rmudambi IHME-Simulationscience 2.0K Mar 17 16:06 ..
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 16:04 tax_dependents_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:59 tax_1040_observer
drwxrwsr-x 11 rmudambi IHME-Simulationscience 4.5K Mar 17 15:53 .
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:51 tax_w2_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:50 social_security_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 wic_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_cps
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:49 household_survey_observer_acs
drwxrwsr-x  2 rmudambi IHME-Simulationscience 9.0K Mar 17 15:48 decennial_census_observer
drwxrwsr-x  2 rmudambi IHME-Simulationscience  18K Mar 17 15:29 logs


In [7]:
!du -s $output_dir

24417532	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [4]:
# Total of 24 GiB in all subdirectories
!du -sh $output_dir

24G	/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19


In [10]:
# 24 GiB = 25.77 GB
2**30/1e9 * 24

25.769803776000003

In [3]:
!ls -halt $output_dir/tax_w2_observer

total 12G
-rw-r--r--  1 rmudambi IHME-Simulationscience 669M Mar 17 15:58 tax_w2_observer_6817.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 665M Mar 17 15:58 tax_w2_observer_4344.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 670M Mar 17 15:58 tax_w2_observer_3541.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 679M Mar 17 15:57 tax_w2_observer_4507.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 674M Mar 17 15:57 tax_w2_observer_6545.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_781.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 677M Mar 17 15:57 tax_w2_observer_9672.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_8869.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 675M Mar 17 15:57 tax_w2_observer_6810.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 673M Mar 17 15:57 tax_w2_observer_1362.csv.bz2
-rw-r--r--  1 rmudambi IHME-Simulationscience 682M Mar 17 15:

In [21]:
!ls -halt $output_dir/logs

total 296K
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:06 make_results_template_seed---seed_4344.o62940658_7
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6810.o62940658_13
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_3541.o62940658_6
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_4507.o62940658_8
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_8869.o62940658_16
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6545.o62940658_12
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:05 make_results_template_seed---seed_6817.o62940658_14
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:04 make_results_template_seed---seed_781.o62940658_15
-rw-r--r--  1 rmudambi IHME-Simulationscience 6.2K Mar 17 16:04 m

# See how `os.walk` works for iterating recursively through files in subdirectories

- https://docs.python.org/3/library/os.html#os.walk
- https://www.reddit.com/r/learnpython/comments/6yzgpm/osscandir_vs_oswalk_any_benefitdifference_of/
- https://stackoverflow.com/questions/10989005/do-i-understand-os-walk-right
- https://stackoverflow.com/questions/16953842/using-os-walk-to-recursively-traverse-directories-in-python

In [14]:
walker = os.walk(output_dir)
walker

<generator object walk at 0x7f5ead522900>

In [15]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19',
 ['decennial_census_observer',
  'tax_1040_observer',
  'wic_observer',
  'logs',
  'social_security_observer',
  'tax_dependents_observer',
  'household_survey_observer_cps',
  'tax_w2_observer',
  'household_survey_observer_acs'],
 [])

In [17]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/decennial_census_observer',
 [],
 ['decennial_census_observer_1362.csv.bz2',
  'decennial_census_observer_6545.csv.bz2',
  'decennial_census_observer_781.csv.bz2',
  'decennial_census_observer_5949.csv.bz2',
  'decennial_census_observer_4507.csv.bz2',
  'decennial_census_observer_3254.csv.bz2',
  'decennial_census_observer_6817.csv.bz2',
  'decennial_census_observer_4344.csv.bz2',
  'decennial_census_observer_6810.csv.bz2',
  'decennial_census_observer_2787.csv.bz2',
  'decennial_census_observer_9672.csv.bz2',
  'decennial_census_observer_5616.csv.bz2',
  'decennial_census_observer_5020.csv.bz2',
  'decennial_census_observer_9901.csv.bz2',
  'decennial_census_observer_3167.csv.bz2',
  'decennial_census_observer_2284.csv.bz2',
  'decennial_census_observer_3541.csv.bz2',
  'decennial_census_obs

In [18]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/tax_1040_observer',
 [],
 ['tax_1040_observer_6810.csv.bz2',
  'tax_1040_observer_9901.csv.bz2',
  'tax_1040_observer_781.csv.bz2',
  'tax_1040_observer_1362.csv.bz2',
  'tax_1040_observer_3254.csv.bz2',
  'tax_1040_observer_2284.csv.bz2',
  'tax_1040_observer_3541.csv.bz2',
  'tax_1040_observer_9672.csv.bz2',
  'tax_1040_observer_6545.csv.bz2',
  'tax_1040_observer_8869.csv.bz2',
  'tax_1040_observer_5020.csv.bz2',
  'tax_1040_observer_2787.csv.bz2',
  'tax_1040_observer_5616.csv.bz2',
  'tax_1040_observer_4344.csv.bz2',
  'tax_1040_observer_6817.csv.bz2',
  'tax_1040_observer_4507.csv.bz2',
  'tax_1040_observer_3167.csv.bz2',
  'tax_1040_observer_5949.csv.bz2'])

In [19]:
next(walker)

('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v1.2_parallelize_business_names/united_states_of_america/2023_03_16_17_21_09/final_results/2023_03_17_15_28_19/wic_observer',
 [],
 ['wic_observer_781.csv.bz2',
  'wic_observer_2284.csv.bz2',
  'wic_observer_5616.csv.bz2',
  'wic_observer_6817.csv.bz2',
  'wic_observer_1362.csv.bz2',
  'wic_observer_4344.csv.bz2',
  'wic_observer_3167.csv.bz2',
  'wic_observer_2787.csv.bz2',
  'wic_observer_6545.csv.bz2',
  'wic_observer_4507.csv.bz2',
  'wic_observer_5949.csv.bz2',
  'wic_observer_8869.csv.bz2',
  'wic_observer_9672.csv.bz2',
  'wic_observer_3541.csv.bz2',
  'wic_observer_6810.csv.bz2',
  'wic_observer_3254.csv.bz2',
  'wic_observer_9901.csv.bz2',
  'wic_observer_5020.csv.bz2'])

# Try loading a sample of each file using `os.walk`

In [24]:
%%time
dfs = {}
nrows = 1000
ext = '.csv.bz2'
for dirpath, dirnames, filenames in os.walk(output_dir):
    observer_name = dirpath[dirpath.rindex('/')+1:]
    if observer_name == 'logs':
        continue
    print("="*50)
    print("Loading data from", observer_name, "\n")
    shards = []
    for filename in filenames:
        if filename.endswith(ext):
            filename_root = filename[:-len(ext)]
            filepath = os.path.join(dirpath, filename)
            print("Loading shard", filename)
            %time shards.append(pd.read_csv(filepath, nrows=nrows))
    if shards:
        print("\nConcatenating data for ", observer_name)
        %time df = pd.concat(shards)
        dfs[observer_name] = df
    print("\n")
dfs.keys()

Loading data from 2023_03_17_15_28_19 



Loading data from decennial_census_observer 

Loading shard decennial_census_observer_1362.csv.bz2
CPU times: user 22.3 ms, sys: 3.4 ms, total: 25.7 ms
Wall time: 25.2 ms
Loading shard decennial_census_observer_6545.csv.bz2
CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 22.5 ms
Loading shard decennial_census_observer_781.csv.bz2
CPU times: user 17.9 ms, sys: 3.38 ms, total: 21.3 ms
Wall time: 21.6 ms
Loading shard decennial_census_observer_5949.csv.bz2
CPU times: user 17.2 ms, sys: 4.32 ms, total: 21.5 ms
Wall time: 21.9 ms
Loading shard decennial_census_observer_4507.csv.bz2
CPU times: user 22.2 ms, sys: 0 ns, total: 22.2 ms
Wall time: 22.5 ms
Loading shard decennial_census_observer_3254.csv.bz2
CPU times: user 21.9 ms, sys: 0 ns, total: 21.9 ms
Wall time: 22.2 ms
Loading shard decennial_census_observer_6817.csv.bz2
CPU times: user 19 ms, sys: 2.52 ms, total: 21.6 ms
Wall time: 21.9 ms
Loading shard decennial_census_observer_434

CPU times: user 20.7 ms, sys: 0 ns, total: 20.7 ms
Wall time: 20.8 ms

Concatenating data for  social_security_observer
CPU times: user 4.37 ms, sys: 0 ns, total: 4.37 ms
Wall time: 4.36 ms


Loading data from tax_dependents_observer 

Loading shard tax_dependents_observer_3254.csv.bz2
CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 24.9 ms
Loading shard tax_dependents_observer_5020.csv.bz2
CPU times: user 24.2 ms, sys: 1.14 ms, total: 25.3 ms
Wall time: 25.5 ms
Loading shard tax_dependents_observer_5616.csv.bz2
CPU times: user 25.5 ms, sys: 0 ns, total: 25.5 ms
Wall time: 25.7 ms
Loading shard tax_dependents_observer_2787.csv.bz2
CPU times: user 22.3 ms, sys: 3.21 ms, total: 25.5 ms
Wall time: 25.8 ms
Loading shard tax_dependents_observer_6810.csv.bz2
CPU times: user 24.9 ms, sys: 0 ns, total: 24.9 ms
Wall time: 25.2 ms
Loading shard tax_dependents_observer_9901.csv.bz2
CPU times: user 25.6 ms, sys: 0 ns, total: 25.6 ms
Wall time: 25.9 ms
Loading shard tax_dependents_obs

CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 22 ms
Loading shard household_survey_observer_acs_781.csv.bz2
CPU times: user 21.5 ms, sys: 0 ns, total: 21.5 ms
Wall time: 21.8 ms
Loading shard household_survey_observer_acs_4344.csv.bz2
CPU times: user 20.8 ms, sys: 562 µs, total: 21.4 ms
Wall time: 21.7 ms
Loading shard household_survey_observer_acs_2787.csv.bz2
CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 21.5 ms
Loading shard household_survey_observer_acs_8869.csv.bz2
CPU times: user 21.7 ms, sys: 0 ns, total: 21.7 ms
Wall time: 22.1 ms
Loading shard household_survey_observer_acs_3541.csv.bz2
CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 22.9 ms
Loading shard household_survey_observer_acs_3167.csv.bz2
CPU times: user 22.9 ms, sys: 0 ns, total: 22.9 ms
Wall time: 23.4 ms

Concatenating data for  household_survey_observer_acs
CPU times: user 9.63 ms, sys: 0 ns, total: 9.63 ms
Wall time: 9.64 ms


CPU times: user 3.38 s, sys: 156 ms, total

dict_keys(['decennial_census_observer', 'tax_1040_observer', 'wic_observer', 'social_security_observer', 'tax_dependents_observer', 'household_survey_observer_cps', 'tax_w2_observer', 'household_survey_observer_acs'])